In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer

In [2]:
# --- A. LOAD DATA AND SPLIT ---
try:
    # Load your encoded and ready-to-scale features
    df_train = pd.read_csv('data/train_processed.csv')
    df_test = pd.read_csv('data/test_processed.csv')
    test_ids = pd.read_csv('data/ids.csv')
except FileNotFoundError:
    print("FATAL ERROR: Could not find files. Please check file paths ('data/').")
    raise

# Define X/y
X_train_full = df_train.drop(['has_copd_risk'], axis=1, errors='ignore')
y_train_full = df_train['has_copd_risk']

# Split for Preprocessor fitting (80% Train, 20% Val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

# --- B. PREPROCESSING PIPELINE (Fit/Transform) ---
all_features = X_train.columns.tolist()

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()) # Scaling is kept for consistency
])

preprocessor = ColumnTransformer(
    transformers=[('num', numerical_pipeline, all_features)],
    remainder='passthrough'
)

# Fit preprocessor on training data and transform it
X_train_processed = preprocessor.fit_transform(X_train)

# ----------------------------------------------------------------------
# C. TRAIN MODEL, PREDICT, AND SAVE SUBMISSION
# ----------------------------------------------------------------------

# 1. Train Random Forest Model
# Using class_weight='balanced' to help handle the F1 score/imbalance
rf_model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1, class_weight='balanced') 
rf_model.fit(X_train_processed, y_train)
print("✅ Random Forest Model trained successfully.")

# 2. Evaluate Baseline F1 Score on Validation Set
X_val_processed = preprocessor.transform(X_val)
y_pred_val = rf_model.predict(X_val_processed)
f1 = f1_score(y_val, y_pred_val)
print(f"Validation F1 Score (Baseline): {f1:.4f}")

# 3. Prepare Test Features and Predict
X_test_full = df_test.drop('patient_id', axis=1, errors='ignore')
X_test_full = X_test_full[all_features] # Ensure column order

# Apply Preprocessor to Test Data (Transform only!)
X_test_processed = preprocessor.transform(X_test_full)

# Prediction
y_test_pred = rf_model.predict(X_test_processed)

# 4. Combine IDs and Predictions and Save
submission_df = test_ids.copy()
submission_df['has_copd_risk'] = y_test_pred.astype(int)

submission_df.to_csv('submission/random_forest.csv', index=False)
print("\n✅ Random Forest submission file saved to 'submission/random_forest.csv'")
print("\nFirst 5 predictions in submission:")
print(submission_df.head())

✅ Random Forest Model trained successfully.
Validation F1 Score (Baseline): 0.7444

✅ Random Forest submission file saved to 'submission/random_forest.csv'

First 5 predictions in submission:
   patient_id  has_copd_risk
0       42427              0
1       27412              0
2       19283              1
3       45261              1
4       11155              1


In [6]:
try:
    # Load your encoded and ready-to-scale features
    df_train = pd.read_csv('data/train_processed.csv')
    df_test = pd.read_csv('data/test_processed.csv')
    test_ids = pd.read_csv('data/ids.csv')
except FileNotFoundError:
    print("FATAL ERROR: Could not find files. Please check file paths ('data/').")
    raise

# Define X/y
X_train_full = df_train.drop(['has_copd_risk'], axis=1, errors='ignore')
y_train_full = df_train['has_copd_risk']

# Split for Preprocessor fitting (80% Train, 20% Val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

# --- B. PREPROCESSING PIPELINE (Fit/Transform) ---
all_features = X_train.columns.tolist()

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[('num', numerical_pipeline, all_features)],
    remainder='passthrough'
)

# Fit preprocessor on training data and transform it
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val) # Transform validation set too

# ----------------------------------------------------------------------
# C. HYPERPARAMETER TUNING (Randomized Search)
# ----------------------------------------------------------------------

# 1. Define Model and Parameter Grid
rf_base = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None], # None means full depth
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['sqrt', 0.5, 0.7]
}

# Use F1 score as the primary metric for optimization
f1_scorer = make_scorer(f1_score)

# 2. Perform Randomized Search CV
# n_iter=20 means it will try 20 random combinations
# cv=3 means 3-fold cross-validation
rf_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_grid,
    n_iter=20,
    scoring=f1_scorer,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit the search to the preprocessed training data
rf_search.fit(X_train_processed, y_train)

# 3. Get the Best Model and Score
best_rf_model = rf_search.best_estimator_
print("\n✅ Hyperparameter Tuning Complete.")
print(f"Best parameters found: {rf_search.best_params_}")

# Evaluate the final best model on the dedicated validation set
y_pred_val = best_rf_model.predict(X_val_processed)
f1_tuned = f1_score(y_val, y_pred_val)
print(f"Validation F1 Score (TUNED): {f1_tuned:.4f}")

# ----------------------------------------------------------------------
# D. PREDICT ON TEST DATA AND SAVE SUBMISSION
# ----------------------------------------------------------------------

# 4. Prepare Test Features
X_test_full = df_test.drop('patient_id', axis=1, errors='ignore')
X_test_full = X_test_full[all_features] # Ensure column order

# Apply Preprocessor to Test Data (Transform only!)
X_test_processed = preprocessor.transform(X_test_full)

# Prediction using the BEST TUNED MODEL
y_test_pred = best_rf_model.predict(X_test_processed)

# 5. Combine IDs and Predictions and Save
submission_df = test_ids.copy()
submission_df['has_copd_risk'] = y_test_pred.astype(int)

# Use 'final_tuned_random_forest' for the output file
submission_df.to_csv('submission/final_tuned_random_forest.csv', index=False) 
print("\n✅ Final submission file saved to 'submission/final_tuned_random_forest.csv'")
print("\nFirst 5 predictions in submission:")
print(submission_df.head())

Fitting 3 folds for each of 20 candidates, totalling 60 fits

✅ Hyperparameter Tuning Complete.
Best parameters found: {'n_estimators': 200, 'min_samples_leaf': 5, 'max_features': 0.7, 'max_depth': 30}
Validation F1 Score (TUNED): 0.7483

✅ Final submission file saved to 'submission/final_tuned_random_forest.csv'

First 5 predictions in submission:
   patient_id  has_copd_risk
0       42427              0
1       27412              0
2       19283              1
3       45261              1
4       11155              1
